# Kyoto Apollonius Circle Map

This notebook generates a map visualizing Kyoto Station, Kiyomizu-dera, and the Apollonius circle defined by a distance ratio between them.

**Parameters:**
- Kyoto Station (A)
- Kiyomizu-dera (B)
- Ratio $k = PA / PB = 2$

In [ ]:
import folium
import numpy as np

# 1. Coordinates
# Kyoto Station (A)
kyoto_station = np.array([34.9858, 135.7588])
# Kiyomizu-dera (B)
kiyomizu_dera = np.array([34.9949, 135.7850])

# 2. Parameters
# Ratio PA / PB = k
# k = 2: Closer to B (Kiyomizu-dera)
k = 2.0

# 3. Calculation of Apollonius Circle
# Internal division point (k:1)
P_in = (kyoto_station + k * kiyomizu_dera) / (1 + k)

# External division point (k:1)
P_out = (kyoto_station - k * kiyomizu_dera) / (1 - k)

# Center and Radius
center = (P_in + P_out) / 2

# Calculate radius in meters for Folium
def get_dist_meters(pt1, pt2):
    R = 6371000 # Earth radius in meters
    lat1, lon1 = np.radians(pt1)
    lat2, lon2 = np.radians(pt2)
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1)*np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    return R * c

radius_meters = get_dist_meters(center, P_in)

# Radius in degrees for generating points
radius_deg_lat = np.abs(P_in[0] - P_out[0]) / 2
radius_deg_lon = radius_deg_lat / np.cos(np.radians(center[0]))

# 4. Visualization
m = folium.Map(location=center, zoom_start=14)

# Marker A: Kyoto Station
folium.Marker(
    kyoto_station, 
    popup='Kyoto Station (A)', 
    icon=folium.Icon(color='blue', icon='train', prefix='fa')
).add_to(m)

# Marker B: Kiyomizu-dera
folium.Marker(
    kiyomizu_dera, 
    popup='Kiyomizu-dera (B)', 
    icon=folium.Icon(color='red', icon='camera', prefix='fa')
).add_to(m)

# Line AB
folium.PolyLine(
    [kyoto_station, kiyomizu_dera], 
    color="black", 
    weight=3, 
    opacity=0.8
).add_to(m)

# Apollonius Circle
folium.Circle(
    location=center,
    radius=radius_meters,
    color='green',
    fill=True,
    fill_opacity=0.1,
    popup=f'Apollonius Circle (k={k})'
).add_to(m)

# Candidate points on the circle
angles = np.linspace(0, 2*np.pi, 5, endpoint=False)
for i, ang in enumerate(angles):
    c_lat = center[0] + radius_deg_lat * np.sin(ang)
    c_lon = center[1] + radius_deg_lon * np.cos(ang)
    
    folium.Marker(
        [c_lat, c_lon], 
        popup=f'Candidate Info Center {i+1}', 
        icon=folium.Icon(color='green', icon='info-sign')
    ).add_to(m)

m